# Step 11 - The trader strategy

Nandita is the Queen of crypto trading - she wants to follow the popular trader's adage of BUY LOW, SELL HIGH

## Nandita's Transaction History
- She also starts out with a 50 BTC and ETH purchase just like Leah and Vikram
- She continues to buy more crypto over the 4 years
- She starts selling some of her crypto portfolio to realise gains

## Nandita's Data
This final scenario 3 is actually exactly the same as our real trading.transactions dataset!

To complete our individual scenarios before we calculate all our metrics for all mentors - let's also prepare another temp table called `nandita_trading_strategy`

In [ ]:
# Importing packages:
import pandas as pd
import os
import sqlalchemy

# Connecting to MySQL database:
host = os.environ.get('mysql_host')
user = os.environ.get('mysql_user')
password = os.environ.get('mysql_password')
engine = sqlalchemy.create_engine(f'mysql+pymysql://{user}:{password}@{host}/trading')

In [ ]:
query = """
SELECT *
FROM transactions
WHERE member_id='a87ff6'
"""
base_table = pd.read_sql_query(query, engine)

base_table.to_sql(
  name='base_table_step11', 
  con=engine,
  index=False,
  if_exists='replace'
  )

You can inspect the data by running the following query after creating the temp table above:

In [ ]:
pd.read_sql(
    """
    SELECT *
    FROM base_table_step11
    LIMIT 10;
    """,
    engine
)

,index,txn_id,member_id,ticker,txn_date,txn_time,txn_type,quantity,percentage_fee
0,0,3,a87ff6,BTC,2017-01-01,2017-01-01 00:00:00,BUY,50.00000,0.00
1,1,19,a87ff6,ETH,2017-01-01,2017-01-01 00:00:00,BUY,50.00000,0.20
2,2,41,a87ff6,ETH,2017-01-01,2017-01-01 17:39:10,BUY,1.98666,0.30
3,3,49,a87ff6,ETH,2017-01-02,2017-01-02 04:48:50,BUY,8.78674,0.30
4,4,53,a87ff6,BTC,2017-01-02,2017-01-02 09:55:27,BUY,5.95981,0.30
5,5,60,a87ff6,BTC,2017-01-02,2017-01-02 17:16:29,BUY,9.01118,0.30
6,6,64,a87ff6,ETH,2017-01-02,2017-01-02 20:49:33,BUY,1.37716,0.01
7,7,77,a87ff6,BTC,2017-01-03,2017-01-03 12:30:20,BUY,3.80769,0.30
8,8,89,a87ff6,BTC,2017-01-04,2017-01-04 08:13:07,BUY,5.68677,0.00
9,9,93,a87ff6,BTC,2017-01-04,2017-01-04 12:25:48,BUY,8.13772,0.30


## Final Evaluation Metrics
By the end of our assessment period on the 29th of August 2021 - we can calculate Nandita's metrics as follows for each individual BTC and ETH portfolio:

- Count of buy and sell transactions
- Total investment amount of purchases
- The dollar amount of fees for purchase transactions
- Dollar cost average of purchases
- Total gross revenue of sell transactions
- Average sell price for each unit sold
- Final portfolio value and quantity
- Profitability measured as (final portfolio value + gross sales revenue - purchase fees - sales fees) / initial investment amount

**Bonus Question**

We also want to calculate the difference if Nandita didn't sell any of her crypto and compare it to the final value at the end of August - how much does this impact her overall profitability?

## Solutions
**Question 1**

Calculate Nandita's purchase metrics for each of her BTC and ETH portfolios:

- Count of purchase transactions
- Initial investment
- Purchase fees
- Dollar cost average of purchases

In [ ]:
pd.read_sql(
    """
    SELECT
      n.ticker,
      COUNT(*) AS purchase_count,
      SUM(n.quantity * p.price) AS initial_investment,
      SUM(n.quantity * p.price * n.percentage_fee / 100) AS purchase_fees,
      SUM(n.quantity * p.price) / SUM(n.quantity) AS dollar_cost_average
    FROM base_table_step11 n
    INNER JOIN prices p
      ON n.ticker=p.ticker
      AND n.txn_date=p.market_date
    WHERE n.txn_type='BUY'
    GROUP BY n.ticker
    ORDER BY purchase_count DESC;
    """,
    engine
)

,ticker,purchase_count,initial_investment,purchase_fees,dollar_cost_average
0,BTC,954,6.373535e+07,162919.941342,12686.918770
1,ETH,756,2.287097e+06,5783.326608,598.397237


## Question 2
Calculate Nandita's sales metrics for each of her BTC and ETH portfolios:

- Count of sales transactions
- Gross revenue amount
- Sales fees
- Average selling price

In [ ]:
pd.read_sql(
    """
   WITH cte_sales AS (
     SELECT
       transactions.ticker,
       COUNT(*) AS sales_count,
       SUM(transactions.quantity) AS sales_quantity,
       SUM(transactions.quantity * prices.price) AS gross_revenue,
       SUM(transactions.quantity * prices.price * transactions.percentage_fee / 100) AS sales_fees
     FROM base_table_step11 AS transactions
    INNER JOIN prices
      ON transactions.ticker = prices.ticker
      AND transactions.txn_date = prices.market_date
    WHERE transactions.txn_type = 'SELL'
    GROUP BY transactions.ticker
    )
    SELECT
      ticker,
      sales_count,
      sales_quantity,
      gross_revenue,
      sales_fees,
      gross_revenue / sales_quantity AS average_selling_price
    FROM cte_sales
    ORDER BY sales_count DESC;
    """,
    engine
)

,ticker,sales_count,sales_quantity,gross_revenue,sales_fees,average_selling_price
0,BTC,167,863.485818,1.097575e+07,29522.092485,12710.973202
1,ETH,70,318.150635,1.725919e+05,447.938134,542.484933


## Question 3
What is Nandita's final BTC and ETH portfolio value and quantity?

In [ ]:
pd.read_sql(
    """
    WITH cte_adjusted_transactions AS (
    SELECT
      member_id,
      txn_date,
      txn_type,
      ticker,
      percentage_fee,
      CASE
        WHEN txn_type = 'BUY' THEN quantity
        WHEN txn_type = 'SELL' THEN -quantity
      END AS quantity
    FROM base_table_step11
    )
    SELECT
      transactions.ticker,
      SUM(transactions.quantity) AS final_quantity,
      SUM(transactions.quantity * prices.price) AS final_portfolio_value
    FROM cte_adjusted_transactions AS transactions
    INNER JOIN trading.prices
      ON transactions.ticker = prices.ticker
    WHERE prices.market_date = '2021-08-29'
    GROUP BY transactions.ticker;
    """,
    engine
)

,ticker,final_quantity,final_portfolio_value
0,BTC,4160.219804,2.007514e+08
1,ETH,3503.886601,1.113479e+07


## Question 4 & 5 (bonus!)
What is Nandita's overall profitability and theoretical profitability if she didn't sell any of her portfolio?

We will try to minimise how many times we access the temp table nandita_trading_strategy to optimise our query performance!

In [ ]:
pd.read_sql(
    """
    WITH cte_portfolio AS (
  SELECT
    transactions.ticker,
    transactions.txn_type,
    COUNT(*) AS transaction_count,
    SUM(transactions.quantity) AS total_quantity,
    SUM(transactions.quantity * prices.price) AS gross_values,
    SUM(transactions.quantity * prices.price * transactions.percentage_fee / 100) AS fees 
  FROM base_table_step11 AS transactions
  INNER JOIN trading.prices
    ON transactions.ticker = prices.ticker
    AND transactions.txn_date = prices.market_date
  GROUP BY 1,2
),
cte_summary AS (
  SELECT
    ticker,
    SUM(
      CASE
        WHEN txn_type = 'BUY' THEN total_quantity
        WHEN txn_type = 'SELL' THEN -total_quantity
      END
    ) AS final_quantity,
    SUM(CASE WHEN txn_type = 'BUY' THEN gross_values ELSE 0 END) AS initial_investment,
    SUM(CASE WHEN txn_type = 'SELL' THEN gross_values ELSE 0 END) AS sales_revenue,
    SUM(CASE WHEN txn_type = 'BUY' THEN fees ELSE 0 END) AS purchase_fees,
    SUM(CASE WHEN txn_type = 'SELL' THEN fees ELSE 0 END) AS sales_fees,
    SUM(CASE WHEN txn_type = 'BUY' THEN total_quantity ELSE 0 END) AS purchase_quantity,
    SUM(CASE WHEN txn_type = 'SELL' THEN total_quantity ELSE 0 END) AS sales_quantity,
    SUM(CASE WHEN txn_type = 'BUY' THEN transaction_count ELSE 0 END) AS purchase_transactions,
    SUM(CASE WHEN txn_type = 'SELL' THEN transaction_count ELSE 0 END) AS sales_transactions
  FROM cte_portfolio
  GROUP BY ticker
),
cte_metrics AS (
  SELECT
    summary.ticker,
    summary.final_quantity * final.price AS actual_final_value,
    summary.purchase_quantity * final.price AS theoretical_final_value,
    summary.sales_revenue,
    summary.purchase_fees,
    summary.sales_fees,
    summary.initial_investment,
    summary.purchase_quantity,
    summary.sales_quantity,
    summary.purchase_transactions,
    summary.sales_transactions,
    summary.initial_investment / purchase_quantity AS dollar_cost_average,
    summary.sales_revenue / sales_quantity AS average_selling_price
  FROM cte_summary AS summary
  INNER JOIN trading.prices AS final
    ON summary.ticker = final.ticker
  WHERE final.market_date = '2021-08-29'
)
SELECT
  ticker,
  actual_final_value AS final_portfolio_value,
  ( actual_final_value + sales_revenue - purchase_fees - sales_fees ) / initial_investment AS actual_profitability,
  ( theoretical_final_value - purchase_fees ) / initial_investment AS theoretical_profitability,
  dollar_cost_average,
  average_selling_price,
  sales_revenue,
  purchase_fees,
  sales_fees,
  initial_investment,
  purchase_quantity,
  sales_quantity,
  purchase_transactions,
  sales_transactions
FROM cte_metrics;
    
    """,
    engine
)

,ticker,final_portfolio_value,actual_profitability,theoretical_profitability,dollar_cost_average,average_selling_price,sales_revenue,purchase_fees,sales_fees,initial_investment,purchase_quantity,sales_quantity,purchase_transactions,sales_transactions
0,ETH,1.113479e+07,4.941266,5.308057,598.397237,542.484933,1.725919e+05,5783.326608,447.938134,2.287097e+06,3822.037236,318.150635,756.0,70.0
1,BTC,2.007514e+08,3.318954,3.800968,12686.918770,12710.973202,1.097575e+07,162919.941342,29522.092485,6.373535e+07,5023.705622,863.485818,954.0,167.0


# References

- [Data With Danny Course - Step 11](https://github.com/DataWithDanny/sql-masterclass/blob/main/course-content/step11.md)